In [1]:
from biofrost.env import *

In [2]:
from tqdm import tqdm
from biofrost.parser import yield_gff, Faidx, revcomp
from collections import defaultdict

gtf_file = '/data/public/database/GENCODE/Mouse_Release_M25/gencode.vM25.annotation.gtf'
genome_file = '/data/public/database/GENCODE/Mouse_Release_M25/GRCm38.p6.genome.fa'

# Load transcriptome structure
tx_structure = defaultdict(dict)
for parser in yield_gff(gtf_file, ):
    if parser.type != 'exon':
        continue
    if parser.gene_type != 'protein_coding':
        continue
    tx_structure[parser.gene_id].setdefault(parser.transcript_id, []).append(parser)

# Load reference genome
genome = Faidx(genome_file)
# aligner = mp.Aligner('/data/public/database/GENCODE/Mouse_Release_M25/GRCm38.p6.genome.fa', preset='splice', n_threads=16)

In [3]:
simulate_fa = './circRNAs.fa'
simulate_abund = './abundances.tsv'

print("Simulating circRNAs")
sim_res = []
for gene_id in tqdm(tx_structure):
    for tx_id, exons in tx_structure[gene_id].items():
        # Exclude the first and last exon for generating circRNAs
        if len(exons) < 3:
            continue
            
        # Skip scaffolds
        if '.' in exons[0].contig:
            continue

        # If generate circRNA
        is_circ = np.random.rand()
        if is_circ < 0.5:
            continue

        # Simulate circRNAs from internal exons
        start_exon, end_exon = sorted(np.random.randint(1, len(exons) - 1, 2))

        # Get sequence
        circ_seq = ''
        sites = []        
        for exon in exons[start_exon:end_exon+1]:
            tmp_seq = genome.seq(exon.contig, exon.start - 1, exon.end)
            if exon.strand == '-':
                tmp_seq = revcomp(tmp_seq)
            circ_seq += tmp_seq
            sites += [exon.start, exon.end]
        circ_id = f'{tx_id}|{gene_id}|{exons[0].contig}:{min(sites)}-{max(sites)}|{exons[0].strand}|{len(circ_seq)}'

        # Random permutation
        start = np.random.randint(0, len(circ_seq))
        pseudo_seq = circ_seq[start:] + circ_seq * 10 # Generate pseudo circular reference
        
        # Random expression levels
        tpm = np.random.beta(0.3, 1)
        sim_res.append((circ_id, circ_seq, pseudo_seq, tpm))

# Expression level Normalization
print("Output simulation results")
factor = 1000 * 1000 / np.sum([i[3] for i in sim_res])
with open(simulate_fa, 'w') as fa, open(simulate_abund, 'w') as tsv:
    for circ_id, circ_seq, pseudo_seq, tpm in tqdm(sim_res):
        fa.write('>{}\n{}\n'.format(circ_id, pseudo_seq))
        tsv.write('{}\t{}\t{}\n'.format(circ_id, circ_seq, tpm * factor))

Simulating circRNAs


100%|██████████| 21859/21859 [00:01<00:00, 15027.97it/s]


Output simulation results


100%|██████████| 41092/41092 [00:00<00:00, 235786.11it/s]


```bash
# Reference genome and annotaion
gtf='/data/public/database/GENCODE/Mouse_Release_M25/gencode.vM25.annotation.gtf'
genome='/data/public/database/GENCODE/Mouse_Release_M25/GRCm38.p6.genome.fa'
transcriptome='/data/public/database/GENCODE/Mouse_Release_M25/gencode.vM25.transcripts.fa'

# Control sequencing dataset
control_fq="./mouse_liver_control.fq.gz"
simulate_circ='./circRNAs.fa'
simulate_abund='./abundances.tsv'
output="./simulated"
threads=16

# Step1. Characterization stage
./NanoSim-2.4-beta/src/read_analysis.py transcriptome \
        -i <(gunzip -c ${control_fq}) \
        -rg ${genome} \
        -rt ${transcriptome} \
        -a minimap2 \
        -o ${output}/control \
        --no_intron_retention \
        -t ${threads}

# Step2. Simulation circRNA reads
./NanoSim-2.4-beta/src/simulator.py transcriptome \
        -rg ${genome} \
        -rt ${simulate_circ} \
        -c ${output}/control \
        -o ${output}/circ \
        -n 200000 \
        -e ${simulate_abund} \
        --no_model_ir

# Step3. Quantification of linear genes
./NanoSim-2.4-beta/src/read_analysis.py quantify \
    -i ${control_fq} \
    -rt ${transcriptome} \
    -t ${threads} \
    -o ${output}/expression

# Step4. simulate linear mRNA reads
./NanoSim-2.4-beta/src/simulator.py transcriptome \
    -rg ${genome} \
    -rt ${transcriptome} \
    -c ${output}/control \
    -o ${output}/linear \
    -n 200000 \
    -e ${output}/expression_abundance.tsv
    --no_model_ir
```